In [ ]:
import torch
from torch import nn, optim
from tqdm import tqdm
from torchvision import datas, transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

%matplotlib inline

In [ ]:
iterations=400000
learning_rate=0.0005
weight_decay=0.01
batch_size=64
vae_input_dim=768
vae_n_cat_feats=0
vae_hidden_dims=[512, 256, 128]
vae_embed_dim=32
vae_codebook_size=256
vae_codebook_normalize=False
vae_sim_vq=False
save_model_every=5000
eval_every=5000
data_folder="data/lfm"
data="data/"
save_dir_root="out/rqvae/lfm/"
wandb_logging=True
commitment_weight=0.25
vae_n_layers=3
vae_codebook_mode="modules.quantize.QuantizeForwardMode.ROTATION_TRICK"
force_data_process=False
data_split="beauty"
do_eval=True